In [1]:
import cv2
import os
import numpy as np
from IPython.display import Image
import imutils
import tqdm
import re

In [36]:
def charger(file):
    img = cv2.imread(file, cv2.IMREAD_UNCHANGED)
    return img

def montrer(img):
    cv2.imshow("image", img)
    k = cv2.waitKey(0)

def inserer(xBegin, yBegin, img, frag, angle=0):
    if angle != 0 and angle != None:
        frag = tourner(frag, angle)
    img = ajouterCoucheAlpha(img)
    
    for x in range(0, frag.shape[0]):
        for y in range(0, frag.shape[1]):
            alpha_actuel = frag[x][y][3]
            if alpha_actuel:
                percentFrag = alpha_actuel/255
                img[xBegin + x, yBegin + y][0] = img[xBegin + x, yBegin + y, 0]*(1-percentFrag) + frag[x, y, 0]*percentFrag
                img[xBegin + x, yBegin + y][1] = img[xBegin + x, yBegin + y, 1]*(1-percentFrag) + frag[x, y, 1]*percentFrag
                img[xBegin + x, yBegin + y][2] = img[xBegin + x, yBegin + y, 2]*(1-percentFrag) + frag[x, y, 2]*percentFrag
            """else:
                img[xBegin + x, yBegin + y][0] = 0
                img[xBegin + x, yBegin + y][1] = 0
                img[xBegin + x, yBegin + y][2] = 0"""
    
    return img

def tourner(img, angle):
    h = img.shape[0]
    w = img.shape[1]
    imgRotated = imutils.rotate(img, angle)
    return imgRotated

def ajouterCoucheAlpha(img):
    if img.shape[2] == 3:
        img = np.pad(img, ((0, 0), (0, 0), (0, 1)), mode='constant', constant_values=0)
    return img

def tournerCentre(image, angle):
    (h, w) = image.shape[:2]
    cX = w/2
    cY = h/2
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    return cv2.warpAffine(image, M, (nW, nH))

In [3]:
img = charger("tp1/Michelangelo_ThecreationofAdam_1707x775.jpg")
print("AVANT :", img.shape)
img = ajouterCoucheAlpha(img)
print("APRES :", img.shape)

AVANT : (775, 1707, 3)
APRES : (775, 1707, 4)


In [4]:
#récupération du tableau de base
angelo = charger("tp1/Michelangelo_ThecreationofAdam_1707x775.jpg")
print(angelo.dtype)

uint8


In [5]:
#récupération d'un fragment
fragment = charger("tp1/frag_eroded/frag_eroded_0.png")
print(fragment.shape)

(68, 89, 4)


In [6]:
blank_img = np.zeros(shape=(775, 1707, 3), dtype=np.uint8)
print(blank_img.shape)
blank_img[:,:,:] = 255
montrer(blank_img)

(775, 1707, 3)


In [7]:
imgTestRed = np.zeros(shape=(50, 50, 3), dtype=np.uint8)
imgTestRed[:,:,0] = 255
montrer(imgTestRed)

In [8]:
#test fonction de rotation
montrer(tourner(fragment, 0))

In [9]:
fragment = charger("tp1/PNG_transparency_demonstration_1.png")
angelo = charger("tp1/Michelangelo_ThecreationofAdam_1707x775.jpg")
montrer(inserer(0, 0, angelo, fragment))
"""for anglee in range(1, 90, 5):
    montrer(inserer(0, 0, angelo, fragment, anglee))"""

l'ANGLEEEE 0


'for anglee in range(1, 90, 5):\n    montrer(inserer(0, 0, angelo, fragment, anglee))'

In [86]:
#----------------------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------Question 1------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------------------------------#
angelo = charger("tp1/Michelangelo_ThecreationofAdam_1707x775.jpg")

canvas = np.zeros(shape=(775, 1707, 4), dtype=np.uint8)
canvas[:,:,:] = 255

fragment = charger("tp1/frag_eroded/frag_eroded_0.png")


canvas = (inserer(0, 0, canvas, fragment))

#convertir fragments.txt en une hmap
hmap = {}
fichier_fragments_infos = open("tp1/fragments.txt", "r")
fragments_infos = fichier_fragments_infos.read()
fichier_fragments_infos.close()
fragments_infos = fragments_infos.split('\n')
for ligne in fragments_infos:
    try:
        ligne_coupee = ligne.split(' ')
        hmap[int(ligne_coupee[0])] = [
            int(ligne_coupee[1]),
            int(ligne_coupee[2]),
            float(ligne_coupee[3])
        ]
    except IndexError:
        pass
    except:
        1/0


canvas = angelo
for nom_fichier in tqdm.tqdm(os.listdir('tp1/frag_eroded/')):
    fragment_index = re.findall(r'\d+', nom_fichier)[0]
    fragment_index = int(fragment_index)
    if fragment_index in list(hmap.keys()):
        fragment = charger(f'tp1/frag_eroded/{nom_fichier}')
        offset_x = -int(fragment.shape[0]/2)
        offset_y = -int(fragment.shape[1]/2)
        try:
            canvas = (inserer(hmap[fragment_index][1]-1 +  + offset_x, hmap[fragment_index][0]-1 + offset_y, canvas, fragment, angle=hmap[fragment_index][2]))
        except:
            pass
    


    #print(re.findall(r'\d+', nom_fichier)[0])
    #print(infos_fragment_actuel)
    #fragment = charger(hmap[][], hmap[][], f'tp1/frag_eroded/{nom_fichier}')
    #canvas = (inserer(x, y, canvas, fragment))

#montrer(angelo)
montrer(canvas)




100%|██████████| 328/328 [00:04<00:00, 68.74it/s] 


In [81]:
#----------------------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------Question 2------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------------------------------#
def comparer(morceau_angelo, fragment, pos_x, pos_y, angle):
    res = 0
    if angle != 0:
        fragment = tourner(fragment, angle)
    for x in range(fragment.shape[0]):
        for y in range(fragment.shape[1]):
            mat_diff = morceau_angelo - fragment
            

    res = mat_diff.sum()
    return res

def evaluer_position(angelo, fragment, pos_x, pos_y, angle):
    return comparer(angelo[pos_x + int(fragment.shape[0]/2) : pos_y + int(fragment.shape[1]/2)], fragment, pos_x, pos_y, angle)
    return 0

#def trouver_position()

fragment = charger('tp1/frag_eroded/frag_eroded_0.png')
evaluer_position(fragment, fragment, 10, 10, 0)


Y = np.arange(16).reshape(4,4)

print(Y)
#print(Y[np.ix_([0,3],[0,3])])

print(Y[0:4, 0:4])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


In [123]:
#----------------------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------Question 2------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------------------------------#
delta_x = 1 # unité : px
delta_y = 1 # unité : px
delta_alpha = 1 # unité : px

# vérifie si le fragment solution est bien localisé
def bien_localise(x, y, alpha, x_barre, y_barre, alpha_barre):
    diff_x = np.abs(x - x_barre)
    diff_y = np.abs(y - y_barre)
    diff_alpha = np.abs(alpha - alpha_barre)

    if (diff_x <= delta_x) and (diff_y <= delta_y) and (diff_alpha <= delta_alpha):
        return True
    return False


# on ouvre les fichiers fragments.txt et solutions.txt, 
fichier_fragments_infos = open("tp1/fragments.txt", "r")
fragments_infos = fichier_fragments_infos.read()
fichier_fragments_infos.close()


fichier_solution_infos = open("tp1/solution.txt", "r")
solution_infos = fichier_solution_infos.read()
fichier_solution_infos.close()


fragments_infos = fragments_infos.split('\n')
solution_infos = solution_infos.split('\n')

#on crée une hmap pour réunir fragments_infos et solutions_infos, puis on appellera bien_localiser sur les valeurs de chaque clé de la hmap.
hmap = {}

for ligne in fragments_infos:
    try:
        ligne_coupee = ligne.split(' ')
        hmap[int(ligne_coupee[0])] = [
            int(ligne_coupee[1]),
            int(ligne_coupee[2]),
            float(ligne_coupee[3])
        ]
    except IndexError:
        pass
    except:
        1/0

for ligne in solution_infos:
    try:
        ligne_coupee = ligne.split(' ')
        if len(ligne_coupee) == 4:
            hmap[int(ligne_coupee[0])] += [
                int(ligne_coupee[1]),
                int(ligne_coupee[2]),
                float(ligne_coupee[3])
            ]
    except IndexError:
        pass
    except:
        1/0


res = 0
for key in hmap.keys():
    if bien_localise(hmap[key][0], hmap[key][3], hmap[key][1], hmap[key][4], hmap[key][2], hmap[key][5]):
        
print(res)


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [ ]:
'''
fichier_fragments_infos = open("tp1/fragments.txt", "r")
fragments_infos = fichier_fragments_infos.read()
fichier_fragments_infos.close()


fichier_solutions_infos = open("tp1/solutions.txt", "r")
solutions_infos = fichier_solutions_infos.read()
fichier_solutions_infos.close()


fragments_infos = fragments_infos.split('\n')
for ligne in fragments_infos:
    try:
        ligne_coupee = ligne.split(' ')
        hmap[int(ligne_coupee[0])] = [
            int(ligne_coupee[1]),
            int(ligne_coupee[2]),
            float(ligne_coupee[3])
        ]
    except IndexError:
        pass
    except:
        1/0
'''